In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import matplotlib.pyplot as plt

# データの読み込み

In [2]:
!pwd
!cd /home/masa1357/git/Credit-Score-Classification-Starter
train_df = pd.read_csv('./data/input/train.csv', index_col=0, low_memory=False)

'pwd' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B
�w�肳�ꂽ�p�X��������܂���B


In [3]:
test_df = pd.read_csv('./data/input/test.csv', index_col=0, low_memory=False)

# 前処理
訓練データとテストデータを結合する

In [4]:
df = pd.concat([train_df.drop(['Poor_Credit_Score'], axis=1), test_df])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 1 to 100000
Data columns (total 26 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  object 
 1   Month                     100000 non-null  object 
 2   Name                      90015 non-null   object 
 3   Age                       100000 non-null  int64  
 4   SSN                       100000 non-null  object 
 5   Occupation                92938 non-null   object 
 6   Annual_Income             100000 non-null  float64
 7   Monthly_Inhand_Salary     84998 non-null   float64
 8   Num_Bank_Accounts         100000 non-null  int64  
 9   Num_Credit_Card           100000 non-null  int64  
 10  Interest_Rate             100000 non-null  int64  
 11  Num_of_Loan               100000 non-null  int64  
 12  Type_of_Loan              88592 non-null   object 
 13  Delay_from_due_date       100000 non-null  int64 

欠損がある列を確認する

In [6]:
nan_rows = df.isna().any(axis=1).sum()
print(nan_rows)
df.isnull().sum()

63217


Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                   7062
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit         2091
Num_Credit_Inquiries         1965
Credit_Mix                  20195
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      8784
Payment_Behaviour               0
Monthly_Balance              1200
dtype: int64

欠損値を置換する

In [7]:
nan_rows = df.isna().any(axis=1).sum()
print(nan_rows)

df.loc[:, 'Monthly_Balance'] = pd.to_numeric(df['Monthly_Balance'], errors='coerce')
std_dev = df['Monthly_Balance'].median()
std_dev
df['Monthly_Balance'].fillna(std_dev, inplace=True)

df.loc[:, 'Monthly_Inhand_Salary'] = pd.to_numeric(df['Monthly_Inhand_Salary'], errors='coerce')
std_dev = df['Monthly_Inhand_Salary'].median()
std_dev
df['Monthly_Inhand_Salary'].fillna(std_dev, inplace=True)

df

63217


,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
ID,,,,,,,,,,,,,,,,,,,,,
1,CUS_0x82bf,January,Laurenced,27,986-44-5074,Manager,56714.52,4561.2100,7,10,...,12.0,Standard,1796.88,27.928768,17 Years and 8 Months,Yes,165.335553,146.184765,High_spent_Medium_value_payments,394.600682
2,CUS_0x82bf,February,Laurenced,28,986-44-5074,Manager,56714.52,4561.2100,7,10,...,13.0,NaN,1796.88,38.999315,17 Years and 9 Months,Yes,165.335553,63.447879,High_spent_Large_value_payments,467.337568
3,CUS_0x82bf,March,NaN,28,986-44-5074,Manager,56714.52,4561.2100,7,10,...,13.0,NaN,1796.88,28.571061,NaN,NM,165.335553,334.670987,Low_spent_Small_value_payments,246.114461
4,CUS_0x82bf,April,Laurenced,28,986-44-5074,Manager,56714.52,3093.7450,7,10,...,13.0,Standard,1796.88,24.294876,17 Years and 11 Months,Yes,165.335553,607.341279,Low_spent_Small_value_payments,336.731225
5,CUS_0x82bf,May,Laurenced,28,986-44-5074,Manager,56714.52,4561.2100,703,10,...,13.0,Standard,1796.88,26.156116,18 Years and 0 Months,Yes,165.335553,111.010314,High_spent_Medium_value_payments,429.775133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,CUS_0x17b5,April,Kiharak,51,714-28-0746,Doctor,110468.49,8934.7075,0,7,...,2.0,NaN,1195.71,28.527993,31 Years and 8 Months,No,52.458223,870.831749,Low_spent_Small_value_payments,260.180778
99997,CUS_0x17b5,May,Kiharak,51,714-28-0746,Doctor,110468.49,8934.7075,0,7,...,2.0,NaN,1195.71,31.683169,31 Years and 9 Months,No,52.458223,126.737867,High_spent_Large_value_payments,954.274659
99998,CUS_0x17b5,June,Kiharak,51,714-28-0746,Doctor,110468.49,8934.7075,0,7,...,2.0,Good,1195.71,36.427377,31 Years and 10 Months,No,52.458223,271.759189,High_spent_Medium_value_payments,819.253337


ここでは、欠損を含む特徴を全て削除し、欠損がない特徴だけを選択します。
（自分で工夫してください。）

また、`Customer_ID` と `SSN` は顧客ごとにユニークな値で、今回の予測には使えないため、選択しません。

In [8]:
df = df[['Month', 'Age', 'Annual_Income','Monthly_Inhand_Salary', 'Num_Bank_Accounts', 
         'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 
         'Delay_from_due_date','Outstanding_Debt', 
         'Credit_Utilization_Ratio', 'Payment_of_Min_Amount', 
         'Total_EMI_per_month', 'Payment_Behaviour','Monthly_Balance']]
df

,Month,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Payment_Behaviour,Monthly_Balance
ID,,,,,,,,,,,,,,,
1,January,27,56714.52,4561.2100,7,10,29,4,40,1796.88,27.928768,Yes,165.335553,High_spent_Medium_value_payments,394.600682
2,February,28,56714.52,4561.2100,7,10,29,4,40,1796.88,38.999315,Yes,165.335553,High_spent_Large_value_payments,467.337568
3,March,28,56714.52,4561.2100,7,10,29,4,40,1796.88,28.571061,NM,165.335553,Low_spent_Small_value_payments,246.114461
4,April,28,56714.52,3093.7450,7,10,29,4,40,1796.88,24.294876,Yes,165.335553,Low_spent_Small_value_payments,336.731225
5,May,28,56714.52,4561.2100,703,10,29,4,36,1796.88,26.156116,Yes,165.335553,High_spent_Medium_value_payments,429.775133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,April,51,110468.49,8934.7075,0,7,7,1,7,1195.71,28.527993,No,52.458223,Low_spent_Small_value_payments,260.180778
99997,May,51,110468.49,8934.7075,0,7,7,1,11,1195.71,31.683169,No,52.458223,High_spent_Large_value_payments,954.274659
99998,June,51,110468.49,8934.7075,0,7,7,1,7,1195.71,36.427377,No,52.458223,High_spent_Medium_value_payments,819.253337


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 1 to 100000
Data columns (total 15 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     100000 non-null  object 
 1   Age                       100000 non-null  int64  
 2   Annual_Income             100000 non-null  float64
 3   Monthly_Inhand_Salary     100000 non-null  float64
 4   Num_Bank_Accounts         100000 non-null  int64  
 5   Num_Credit_Card           100000 non-null  int64  
 6   Interest_Rate             100000 non-null  int64  
 7   Num_of_Loan               100000 non-null  int64  
 8   Delay_from_due_date       100000 non-null  int64  
 9   Outstanding_Debt          100000 non-null  float64
 10  Credit_Utilization_Ratio  100000 non-null  float64
 11  Payment_of_Min_Amount     100000 non-null  object 
 12  Total_EMI_per_month       100000 non-null  float64
 13  Payment_Behaviour         100000 non-null  object

欠損がないことを確認しておきます。

In [10]:
df.isnull().sum()

Month                       0
Age                         0
Annual_Income               0
Monthly_Inhand_Salary       0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Delay_from_due_date         0
Outstanding_Debt            0
Credit_Utilization_Ratio    0
Payment_of_Min_Amount       0
Total_EMI_per_month         0
Payment_Behaviour           0
Monthly_Balance             0
dtype: int64

文字列の特徴をダミー変数に変換します。

In [11]:
df = pd.get_dummies(df)
df

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Outstanding_Debt,Credit_Utilization_Ratio,...,Payment_of_Min_Amount_NM,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes,Payment_Behaviour_!@9#%8,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments
ID,,,,,,,,,,,,,,,,,,,,,
1,27,56714.52,4561.2100,7,10,29,4,40,1796.88,27.928768,...,False,False,True,False,False,True,False,False,False,False
2,28,56714.52,4561.2100,7,10,29,4,40,1796.88,38.999315,...,False,False,True,False,True,False,False,False,False,False
3,28,56714.52,4561.2100,7,10,29,4,40,1796.88,28.571061,...,True,False,False,False,False,False,False,False,False,True
4,28,56714.52,3093.7450,7,10,29,4,40,1796.88,24.294876,...,False,False,True,False,False,False,False,False,False,True
5,28,56714.52,4561.2100,703,10,29,4,36,1796.88,26.156116,...,False,False,True,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,51,110468.49,8934.7075,0,7,7,1,7,1195.71,28.527993,...,False,True,False,False,False,False,False,False,False,True
99997,51,110468.49,8934.7075,0,7,7,1,11,1195.71,31.683169,...,False,True,False,False,True,False,False,False,False,False
99998,51,110468.49,8934.7075,0,7,7,1,7,1195.71,36.427377,...,False,True,False,False,False,True,False,False,False,False


In [12]:
nrow, ncol = train_df.shape
print(nrow, ncol)

70000 27


In [13]:
train_df = df[:nrow].join(train_df['Poor_Credit_Score'])

In [14]:
test_df = df[nrow:]

# 可視化
ここでは、相関係数行列を可視化しただけで、予測モデルには何も反映させていません。
（自分で工夫してください。）

In [15]:
import seaborn as sns

In [16]:
# 相関行列を計算
corr = train_df.corr()

# 最後の列との相関を取得し、値によって並べ替え
sorted_corr = corr.iloc[:-1, -1].sort_values(ascending=False)

# 並べ替えた相関を表示
print(sorted_corr)

Delay_from_due_date                                   0.375525
Outstanding_Debt                                      0.364025
Payment_of_Min_Amount_Yes                             0.249492
Payment_Behaviour_Low_spent_Small_value_payments      0.085370
Num_Bank_Accounts                                     0.011550
Payment_Behaviour_Low_spent_Medium_value_payments     0.010087
Month_May                                             0.005740
Month_June                                            0.005359
Interest_Rate                                         0.005064
Num_of_Loan                                           0.003936
Num_Credit_Card                                       0.003143
Month_April                                           0.002787
Payment_Behaviour_!@9#%8                              0.000246
Month_July                                           -0.000262
Month_August                                         -0.000453
Total_EMI_per_month                                  -0

In [17]:
# 相関行列を計算
corr = train_df.corr()

# 最後の列との相関を取得
corr_with_last_column = corr.iloc[:-1, -1]

# 相関の絶対値が0.1より小さい列を抽出
low_corr_columns = corr_with_last_column[corr_with_last_column.abs() < 0].index

print(low_corr_columns)
# データフレームから低相関の列を削除
#train_df = train_df.drop(low_corr_columns, axis=1)
#test_df = test_df.drop(low_corr_columns, axis=1)


Index([], dtype='object')


# 機械学習
ランダムフォレストでグリッドサーチします．

In [18]:
X_train = train_df.drop(['Poor_Credit_Score'], axis=1).to_numpy()
y_train = train_df['Poor_Credit_Score'].to_numpy()

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

# 初期化
mlp = MLPClassifier()

# ハイパーパラメータの範囲を設定
parameters = {
    'hidden_layer_sizes': [(50,), (100,), (150,)],      # 隠れ層のノード数
    'activation'        : ['logistic', 'tanh', 'relu'], # 活性化関数
    'solver'            : ['sgd', 'adam'],              # 最適化のアルゴリズム
    'alpha'             : [0.0001, 0.001, 0.01, 0.1],   # L2ペナルティ（正則化項）のパラメータ
    'learning_rate'     : ['constant', 'adaptive'],     # 学習率のスケジューリング
    'max_iter'          : [500, 1000, 1500],
}

# GridSearchCVのインスタンスを作成
clf = GridSearchCV(mlp, parameters, cv=5)
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print(clf.best_params_)


In [ ]:
s = model.score(X_train, y_train)
print('平均正解率:', s)

平均正解率: 0.7501285714285715


In [ ]:
from sklearn.model_selection import KFold, cross_val_score
kfold = KFold(n_splits=10)
scores = cross_val_score(model, X_train, y_train, cv=kfold)
print("K-Fold CV Score: ", scores.mean())

K-Fold CV Score:  0.7052285714285714


# 予測
学習したモデルを用いて、テストデータの `Poor_Credit_Score` を予測する。
評価尺度が **AUC** であるため、`predict()` の替わりに `predict_proba()` を用いて確率を出力する。

In [ ]:
X_test = test_df.to_numpy()

p_test =model.predict_proba(X_test)
p_test

array([[1.00000000e+00, 5.67802190e-28],
       [1.00000000e+00, 1.07865629e-21],
       [1.00000000e+00, 1.65066275e-32],
       ...,
       [1.00000000e+00, 8.92121435e-60],
       [1.00000000e+00, 3.02347996e-61],
       [1.00000000e+00, 1.91563374e-32]])

`Poor_Credit_Score` の値が 0 になる確率、1 になる確率が出力されているので、1 になる確率を取り出して提出用データに代入する。

In [ ]:
submit_df = pd.read_csv('./data/output/sampleSubmission.csv', index_col=0)
submit_df['Poor_Credit_Score'] = p_test[:,1]
submit_df

,Poor_Credit_Score
ID,
70001,5.678022e-28
70002,1.078656e-21
70003,1.650663e-32
70004,7.323023e-32
70005,3.926382e-29
...,...
99996,1.422035e-34
99997,1.205674e-60
99998,8.921214e-60


CSVファイルとして出力する。

In [ ]:
submit_df.to_csv('./data/output/sampleSubmission.csv')

<h2>評価</h2>
leave-one-out cross-validation, LOO-CV で評価